## Informe de Extracción, Transformación y Carga de Datos: ETL_user_reviews

En esta sección, nos sumergiremos en el proceso de Extracción, Transformación y Carga (ETL) de los datos provenientes del conjunto de reseñas de usuarios australianos de los juegos en la plataforma **Steam**. El objetivo principal es preparar estos datos para su análisis, asegurándonos de que sean aptos y consistentes.

Comenzaremos importando las bibliotecas esenciales. Asegúrese de tener estas bibliotecas instaladas previamente para garantizar una ejecución sin contratiempos.

A lo largo del informe, nos enfocaremos en abordar problemas potenciales en los datos, aplicar técnicas de limpieza y preprocesamiento, y finalmente, almacenar los datos transformados para futuras exploraciones y análisis.

### Requisitos


⚠️ **Asegúrese de instalar las siguientes bibliotecas antes de ejecutar el código**

- pandas
- numpy
- gdown
- langdetect
- nltk

Puede instalar estas bibliotecas debe abrir una terminal o ventana de línea de comandos y ejecutar el siguiente comando:

*`pip install pandas numpy gdown langdetect nltk`*

### Importar Bibliotecas

In [1]:
# Importamos pandas para el análisis de datos tabulares
import pandas as pd

# NumPy proporciona soporte para arreglos y matrices multidimensionales
import numpy as np

# El módulo os permite interactuar con el sistema operativo
import os

# Gdown facilita la descarga de archivos desde Google Drive utilizando su ID
import gdown

# JSON es un formato común para el intercambio de datos, y Python tiene soporte incorporado para trabajar con JSON
import json

# Langdetect es una biblioteca para detectar automáticamente el idioma en el que está escrito un texto
from langdetect import detect

# NLTK (Natural Language Toolkit) es una plataforma para construir programas Python para trabajar con datos de lenguaje humano
import nltk

# SentimentIntensityAnalyzer es una herramienta en NLTK para análisis de sentimientos
from nltk.sentiment import SentimentIntensityAnalyzer

⚠️**Nota: Este comando descarga el modelo de análisis de sentimiento de NLTK y solo debe ejecutarse la primera vez que se utiliza.**

In [2]:
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\johan\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

A continuación, realizaremos las etapas del proceso ETL:

### 1. Cargar el conjunto de datos original

Fuente de datos: **australian_user_reviews_repair.json**

Es importante señalar que el archivo inicial presentaba inconsistencias en su formato JSON, lo que llevó a su corrupción. Para resolver este problema, utilicé las herramientas "codebeautify" y "notepad++" para reparar el archivo. La versión reparada ahora se encuentra almacenada en Google Drive con acceso compartido. Puede **descargar automáticamente** el archivo ejecutando las siguientes líneas de código (recomendado).

En caso de preferirlo, puede acceder al enlace a continuación para descargar manualmente el archivo: 
Datasets: https://bit.ly/47J98PN

In [3]:
def descargar_leer_json(url, nombre_archivo):
    """
    Descarga un archivo JSON desde la URL proporcionada, lo guarda localmente
    con el nombre especificado y lo lee en un DataFrame de pandas.

    Parameters:
    - url (str): URL del archivo en Google Drive.
    - nombre_archivo (str): Nombre del archivo.

    Returns:
    - df (pd.DataFrame): DataFrame de pandas con los datos del archivo JSON.
    """

    # Verificar si el archivo ya existe localmente (Asegúrese que sea la versión reparada)
    if not os.path.exists(nombre_archivo):
        # Si no existe, descargar el archivo desde Google Drive
        gdown.download(url, nombre_archivo, quiet=False)

    # Lectura del JSON en un DataFrame de pandas
    df = pd.read_json(nombre_archivo)

    return df

In [4]:
#URL del archivo en Google Drive y nombre del archivo local
url = 'https://bit.ly/49LHpQo'
output = 'australian_user_reviews_repair.json'

#Utilizar la función (descargar_leer_json) para descargar y leer el JSON
df_UserReviews = descargar_leer_json(url, output)

Downloading...
From: https://bit.ly/49LHpQo
To: C:\Users\johan\Bootcamp_SoyHenry\PI_MlopsSteam\PI_01\australian_user_reviews_repair.json
100%|██████████| 26.3M/26.3M [01:53<00:00, 232kB/s] 


### 2. Explorar y entender el conjunto de datos

Exploramos y entendemos la estructura del conjunto de datos, revisando las primeras filas, información general y estadísticas descriptivas.

In [5]:
# Mostrar las primeras filas del DataFrame
print("Primeras filas del DataFrame:")
df_UserReviews.head()

Primeras filas del DataFrame:


,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."


La visualización de las primeras 5 filas del conjunto de datos muestra tres columnas principales: 'user_id', 'user_url', y 'reviews'. Aquí hay algunos comentarios al respecto:

- user_id: Esta columna parece contener identificadores únicos para cada usuario. 

- user_url: Contiene enlaces a los perfiles de los usuarios en la plataforma Steam. 

- reviews: Esta columna parece contener información detallada sobre las revisiones realizadas por cada usuario. Cada entrada es un diccionario o estructura similar, que incluye detalles como la fecha de publicación, contenido de la revisión, etc.

Definitivamente la columna 'reviews' contiene datos estructurados (formato JSON). Se procede abrir el archivo en modo lectura con codificación UTF-8 y se carga el contenido como un objeto Python lo que nos permitirá explorar esta columna más a fondo (puede revelar información adicional sobre las revisiones de los usuarios). 

In [6]:
# Abre el archivo 'australian_user_reviews.json' en modo lectura con codificación UTF-8.

# Utiliza el módulo json para cargar el contenido del archivo en un objeto Python.
with open('australian_user_reviews_repair.json', 'r', encoding='utf-8') as file:
    URev = json.loads(file.read())

In [7]:
# Aplana la estructura JSON contenida en URev, extrayendo las reseñas que están anidada bajo la clave 'reviews'.

# Crea un nuevo DataFrame (df_UserReviews) con detalles de cada revisión, manteniendo las columnas 'user_id' y 'user_url'.
df_UserReviews = pd.json_normalize(URev, record_path=['reviews'], meta=['user_id', 'user_url'])

# Lista con el orden deseado de las columnas
column_order = ['user_id', 'user_url', 'funny', 'posted', 'last_edited', 'item_id', 'helpful', 'recommend', 'review']

# Reordena las columnas del DataFrame
df_UserReviews = df_UserReviews[column_order]

df_UserReviews.head()

,user_id,user_url,funny,posted,last_edited,item_id,helpful,recommend,review
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.
2,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted April 21, 2011.",,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...
3,js41637,http://steamcommunity.com/id/js41637,,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...
4,js41637,http://steamcommunity.com/id/js41637,,"Posted September 8, 2013.",,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...


In [8]:
# Obtener información general del DataFrame
print("\nInformación general del DataFrame:")
df_UserReviews.info()


Información general del DataFrame:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59305 entries, 0 to 59304
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      59305 non-null  object
 1   user_url     59305 non-null  object
 2   funny        59305 non-null  object
 3   posted       59305 non-null  object
 4   last_edited  59305 non-null  object
 5   item_id      59305 non-null  object
 6   helpful      59305 non-null  object
 7   recommend    59305 non-null  bool  
 8   review       59305 non-null  object
dtypes: bool(1), object(8)
memory usage: 3.7+ MB


Considerando que las columnas principalmente contienen tipos de datos 'object' y 'bool', resulta menos significativo aplicar estadísticas descriptivas a todas las columnas. No obstante, en relación con la columna 'recommend', observaremos sus valores para obtener una perspectiva clara de la distribución de recomendaciones en el conjunto de datos. 

In [9]:
# Calcula valores en la columna 'recommend' 
conteo_por_recomendaciones = df_UserReviews['recommend'].value_counts()
porcentaje_recomendaciones = df_UserReviews['recommend'].value_counts(normalize=True) * 100
#porcentaje_recomendaciones = porcentaje_recomendaciones.round(2).astype(str) + '%'

# Crear un nuevo DataFrame con el conteo y porcentaje
resumen_recomendaciones = pd.DataFrame({
    'Conteo': conteo_por_recomendaciones,
    'Porcentaje': porcentaje_recomendaciones.round(2).astype(str) + '%'
})

# Ordenar el DataFrame por el conteo de mayor a menor
resumen_recomendaciones = resumen_recomendaciones.sort_values(by='Conteo', ascending=False)
print(resumen_recomendaciones)

       Conteo Porcentaje
True    52473     88.48%
False    6832     11.52%


### 3. Limpiar y preprocesar el conjunto de datos

- Manejar valores nulos o duplicados.
- Identificar y eliminar columnas irrelevantes (si es necesario).
- Realizar análisis de texto para determinar el idioma de las reseñas.
- Realizar análisis de sentimiento para categorizar las reseñas.
- Crear nuevas columnas derivadas de la información existente para facilitar análisis futuros

In [10]:
# Reemplazar valores vacíos, 'null' y 'None' con NaN en todo el DataFrame
df_UserReviews.replace(['', 'null', 'None'], np.nan, inplace=True)

In [11]:
# Filas donde todas las columnas especificadas tienen valores nulos.
columnas_a_considerar = ['posted', 'recommend', 'review']
filas_con_nulos = df_UserReviews[df_UserReviews[columnas_a_considerar].isnull().all(axis=1)]
filas_con_nulos

,user_id,user_url,funny,posted,last_edited,item_id,helpful,recommend,review


Las 'columnas_a_considerar' representan aspectos críticos de las revisiones de usuarios, y la ausencia simultánea de datos en estas áreas podría afectar negativamente la calidad de nuestros resultados.

Los resultados indican que dentro del conjunto de datos actual, no hay filas donde 'posted', 'recommend', y 'review' estén simultáneamente ausentes. Como resultado, podemos proceder con confianza en nuestro análisis.

In [12]:
# Se establece umbral del 80% para decidir que columnas eliminar por valores nulos
umbral_nulos = 0.8

# Calcula el porcentaje de valores nulos por columna
porcentaje_nulos = df_UserReviews.isnull().mean()

# Filtra las columnas que superan el umbral
columnas_a_eliminar = porcentaje_nulos[porcentaje_nulos > umbral_nulos]

# Muestra las columnas y su respectivo porcentaje de valores nulos
print("Columnas con más del {}% de valores nulos (candidatas a eliminar):".format(umbral_nulos * 100))
for columna, porcentaje in columnas_a_eliminar.items():
    print("{}: {:.2%}".format(columna, porcentaje))

Columnas con más del 80.0% de valores nulos (candidatas a eliminar):
funny: 86.26%
last_edited: 89.65%


La mayoría de las filas en las columnas 'funny' y 'last_edited' tienen valores nulos y no son cruciales para nuestro análisis. Adicionalmente, las columnas 'helpful'y 'user_url' tampoco aportan datos relevantes para nuestro estudio objetivo.

Por lo anteriormente justificado se procede eliminar esas cuatro columnas.

In [13]:
# Eliminamos las columnas que pasan el umbral establecido para valores nulos 
df_UserReviews.drop(columns=columnas_a_eliminar.index, inplace=True)

In [14]:
# Se eliminan las columnas 'helpful' y 'user_url' por considerarse no relevantes
df_UserReviews.drop(['helpful','user_url'], axis=1, inplace=True)

In [15]:
# Se buscan registros duplicados
df_UserReviews.sort_values('user_id')
filas_duplicadas = df_UserReviews[df_UserReviews.duplicated(subset=['user_id', 'item_id', 'posted', 'review'], keep=False)]
filas_duplicadas.count()

user_id      1734
posted       1734
item_id      1734
recommend    1734
review       1734
dtype: int64

In [16]:
# Se eliminan los registros duplicados basándome en múltiples columnas 
duplicados_eliminados = df_UserReviews.drop_duplicates(subset=['user_id', 'item_id', 'posted', 'review'], keep=False, inplace=True)

cantidad_total_duplicados_eliminados = filas_duplicadas.shape[0]

print("Cantidad total de registros duplicados eliminados:", cantidad_total_duplicados_eliminados)

Cantidad total de registros duplicados eliminados: 1734


Dentro del análisis de los reviews, es crucial conocer el año en que fueron realizados. Aunque en mi conjunto de datos no contamos con una columna explícita de año, disponemos de la información en la columna 'posted'. 

En consecuencia, se ha decidido crear una nueva columna llamada 'year', de tipo entero, extrayendo la información de la columna 'posted'. Posteriormente, la columna 'posted' se eliminará para simplificar la estructura del conjunto de datos. Este proceso asegura que tengamos un atributo específico para el año de cada review, facilitando análisis temporales y tendencias a lo largo del tiempo.

In [17]:
# Visualizar los datos de la columna 'posted'
unique_posted_values = df_UserReviews['posted'].unique()

for value in unique_posted_values:
    print(value)

Posted November 5, 2011.
Posted July 15, 2011.
Posted April 21, 2011.
Posted June 24, 2014.
Posted September 8, 2013.
Posted November 29, 2013.
Posted February 3.
Posted December 4, 2015.
Posted November 3, 2014.
Posted October 15, 2014.
Posted October 14, 2013.
Posted July 28, 2012.
Posted June 2, 2012.
Posted June 29, 2014.
Posted November 22, 2012.
Posted February 23, 2012.
Posted April 15, 2014.
Posted December 23, 2013.
Posted March 14, 2014.
Posted July 11, 2013.
Posted May 5, 2014.
Posted December 24, 2012.
Posted October 21, 2012.
Posted March 20, 2012.
Posted March 9, 2012.
Posted May 20.
Posted July 24.
Posted February 1, 2015.
Posted June 20, 2014.
Posted June 11.
Posted August 25, 2014.
Posted December 25, 2013.
Posted June 23, 2012.
Posted September 5, 2015.
Posted March 30, 2015.
Posted February 19, 2014.
Posted July 14, 2014.
Posted April 27, 2013.
Posted July 20, 2015.
Posted November 4, 2013.
Posted July 12, 2013.
Posted August 19, 2012.
Posted June 19, 2015.
Posted Se

In [18]:
# Asegúrate de que 'posted' sea de tipo datetime
df_UserReviews['posted'] = pd.to_datetime(df_UserReviews['posted'].astype(str).str.replace(r'Posted |,|\.', '', regex=True), errors='coerce')

# Crea la columna 'year' a partir de 'posted'
df_UserReviews['year'] = df_UserReviews['posted'].dt.year.astype('Int64')

# Ordena el DataFrame por 'item_id' y 'year' para asegurar que la interpolación se haga correctamente
df_UserReviews = df_UserReviews.sort_values(['item_id', 'year'])

In [19]:
# Rellenar valores nulos en 'year' mediante interpolación lineal por grupo (item_id)
df_UserReviews['year'] = df_UserReviews.groupby('item_id', group_keys=False)['year'].apply(lambda group: group.interpolate(method='pad') if group.notna().any() else group)

# Si aún hay valores nulos después de la interpolación, se llenan con la mediana. 
df_UserReviews['year'] = df_UserReviews['year'].fillna(df_UserReviews['year'].median())

In [20]:
# Se eliminan las columnas 'posted' y 'user_id' que ya no son de utilidad
df_UserReviews.drop(['posted', 'user_id'], axis=1, inplace=True)
df_UserReviews.head()

,item_id,recommend,review,year
5331,10,True,this game is the 1# online action game is awes...,2011
22702,10,True,GYERTEK GAMELNI MINDENKI ITT VAN AKI SZÁMIT !!...,2012
35539,10,True,:D,2012
43134,10,True,Good Game :D,2012
24137,10,True,jueguenlooooooo,2013


Identificamos que la columna 'posted' contenía algunos valores estaban ausentes en cuanto al año. Dado que la temporalidad es esencial para nuestro análisis, decidimos abordar este problema de manera estratégica una vez creada la columna 'year'.

- Selección de Técnica de Imputación:
Optamos por utilizar la técnica de interpolación para llenar los valores faltantes en 'year'. La interpolación es útil cuando existe una relación secuencial o temporal en los datos.

- Agrupación por Juego ('item_id'):
Dado que cada review está asociado a un juego único, agrupamos los datos por 'item_id' para considerar la relación temporal dentro de cada juego.

- Aplicación de Interpolación Lineal por Grupo:
Aplicamos la interpolación lineal a cada grupo de 'item_id', lo que permitió estimar los años faltantes basándonos en los años conocidos del mismo juego. Se consideró la opción de aplicar la interpolación lineal global, pero optamos por la interpolación por grupo para capturar posibles variaciones en la temporalidad entre diferentes juegos).


- Manejo de Valores Nulos Restantes:
Después de la interpolación, si aún había valores nulos, llenamos esos espacios con la mediana de la columna 'year'.
Este proceso asegura que nuestro conjunto de datos mantenga la coherencia temporal necesaria para análisis posteriores, y la elección de la interpolación por grupo se alinea con la naturaleza de los datos, donde la temporalidad puede variar entre diferentes juegos. NOTA: Para una 2da versión los registos nulos restantes se podrían llenar con inicialmente con el año de publicación del juego 'release_date' en el archivo steam_games.json y por último con la mediana.

Ahora, evaluaremos la columna 'review' para obtener estadísticas sobre la cantidad de reseñas por lenguaje. Este análisis será crucial para determinar si el proceso de análisis de sentimientos se realizará en todo el conjunto de datos o si se aplicará un filtro específico por idioma.  

In [21]:
# Análisis de texto para determinar el idioma de las reseñas

def detectar_idioma(texto):
    try:
        return detect(texto)
    except:
        return None

# Aplicar la función para detectar idioma y crear una nueva columna 'language'
df_UserReviews['language'] = df_UserReviews['review'].apply(detectar_idioma)

# Calcular el conteo y porcentaje de cada idioma
conteo_por_idioma = df_UserReviews['language'].value_counts()
porcentaje_por_idioma = df_UserReviews['language'].value_counts(normalize=True) * 100

# Crear un nuevo DataFrame con el conteo y porcentaje
resumen_idiomas = pd.DataFrame({
    'Conteo': conteo_por_idioma,
    'Porcentaje': porcentaje_por_idioma.round(2).astype(str) + '%'
})

# Ordenar el DataFrame por el conteo de mayor a menor
resumen_idiomas = resumen_idiomas.sort_values(by='Conteo', ascending=False)
resumen_idiomas.head()

,Conteo,Porcentaje
en,44396,77.87%
pt,2124,3.73%
es,1250,2.19%
de,1135,1.99%
so,1004,1.76%


In [22]:
#esto es para tener la información en el EDA

# Mapeo de códigos de idioma a nombres completos (Top 5)
mapeo_idiomas = {
    'en': 'English',
    'pt': 'Portugués',
    'es': 'Español',
    'de': 'German',
    'so': 'Somali',
    # Agrega más mapeos según sea necesario
}

# Aplicar el mapeo al DataFrame
df_UserReviews['idioma_completo'] = df_UserReviews['language'].map(mapeo_idiomas)

# Crear un DataFrame con el resumen de idiomas
resumen_idiomas = df_UserReviews['idioma_completo'].value_counts().reset_index()
resumen_idiomas.columns = ['Idioma', 'Conteo']
resumen_idiomas['Conteo'] = resumen_idiomas['Conteo'].round(2)

# Calcular el porcentaje
resumen_idiomas['Porcentaje'] = (resumen_idiomas['Conteo'] / len(df_UserReviews)) * 100
resumen_idiomas['Porcentaje'] = resumen_idiomas['Porcentaje'].round(2)

# Guardar en un archivo CSV
resumen_idiomas.to_csv('resumen_idiomas.csv', index=False)


In [23]:
# Filtrar los registros donde language no es igual a 'en' 
df_UserReviews = df_UserReviews[df_UserReviews['idioma_completo'] == 'English']

# Borrar la columna 'idioma_completo'
df_UserReviews = df_UserReviews.drop('idioma_completo', axis=1)

Con los resultados obtenidos consideré realizar el análisis de sentimiento solo en los registros en inglés. Razones que justifican esta decisión (en el EDA se profundiza un poco más):

- Mayor Representatividad.
- Precisión del Modelo.
- Eficiencia Computacional.

**Feature Engineering**

La columna 'review' también será parte de nuestro estudio ya que incluye reseñas de juegos hechos por distintos usuarios. 
Partiendo de 'review' se va crear la columna 'sentiment_analysis' aplicando análisis de sentimiento con NLP con la siguiente escala: Debe tomar el valor '0' si es malo, '1' si es neutral y '2' si es positivo. De no ser posible este análisis por estar ausente la reseña escrita, tomará el valor de 1. 
Esta nueva columna reemplaza la de 'review' para facilitar el trabajo de los modelos de machine learning y el análisis de datos. 

In [24]:
#Se convierten todas las letras a minúsculas para asegurar que todas las palabras sean tratadas de la misma manera.
df_UserReviews.loc[:, 'review'] = df_UserReviews['review'].str.lower()

In [25]:
#Eliminación de caracteres especiales
df_UserReviews['review'] = df_UserReviews['review'].replace('[^A-Za-z0-9\s]+', '', regex=True)

In [26]:
#Elimina caracteres de puntuación que no aportan al análisis de sentimiento.
df_UserReviews.loc[:, 'review'] = df_UserReviews['review'].str.replace('[^\w\s]', '', regex=True)

In [27]:
# Análisis de sentimiento para categorizar las reseñas columna 'review' 

def analyze_sentiments(df):
    # Instanciar el analizador de sentimientos
    sia = SentimentIntensityAnalyzer()

    # Aplicar el análisis de sentimientos y asignar valores numéricos
    df['compound_score'] = df['review'].apply(lambda review: sia.polarity_scores(review)['compound'])
    df['sentiment_analysis'] = df['compound_score'].apply(lambda score: 0 if score < 0 else (1 if score == 0 else 2))

    # Conteo de reviews por score
    score_counts = df['sentiment_analysis'].value_counts()

    # Conteo de reviews en blanco
    blank_reviews_count = df['review'].isnull().sum()

    # Total de reviews
    total_reviews = len(df)

    # Calcular porcentajes
    score_percentages = (score_counts / total_reviews * 100).round(2)
    blank_reviews_percentage = (blank_reviews_count / total_reviews * 100).round(2)
     
    # Se eliminan las columnas 'review' y 'compound_score', no necesitaremos estos datos
    df.drop(['review','compound_score'], axis=1, inplace=True)  

    return df, score_counts, blank_reviews_count, score_percentages, blank_reviews_percentage

In [28]:
# Llamar a la función analyze_sentiments
df_UserReviews, score_counts, blank_reviews_count, score_percentages, blank_reviews_percentage = analyze_sentiments(df_UserReviews)

# Crear un nuevo DataFrame con el conteo y porcentaje
resumen_sentimientos = pd.DataFrame({
    'Conteo': score_counts,
    'Porcentaje': score_percentages.round(2).astype(str) + '%'
})

# Ordenar el DataFrame por el conteo de mayor a menor
resumen_sentimientos = resumen_sentimientos.sort_values(by='Conteo', ascending=False)

# Imprimir los resultados
print("\nResumen de análisis de sentimientos:")
print(resumen_sentimientos)
print("\nConteo de reviews en blanco: ", blank_reviews_count, " Porcentaje: ", blank_reviews_percentage.round(2).astype(str) + '%')


Resumen de análisis de sentimientos:
   Conteo Porcentaje
2   31449     70.84%
0    8269     18.63%
1    4678     10.54%

Conteo de reviews en blanco:  0  Porcentaje:  0.0%


In [30]:
# Borrar la columna 'language'
df_UserReviews = df_UserReviews.drop('language', axis=1)
df_UserReviews

,item_id,recommend,year,sentiment_analysis
5331,10,True,2011,2
45506,10,True,2013,1
7801,10,True,2014,2
7967,10,True,2014,2
8519,10,True,2014,2
...,...,...,...,...
51725,99900,True,2015,2
53065,99900,False,2015,2
12393,99910,True,2011,2
53052,99910,False,2014,0


Al seleccionar un método de análisis de sentimientos es fundamental considerar varios factores, como la precisión del método, la facilidad de implementación y la eficiencia computacional. Se optó por utilizar la biblioteca NLTK (Natural Language Toolkit). Se recomienda revisar el EDA donde se profundiza un poco más sobre el tema.

Justificación:
- Facilidad de Implementación.
- Eficiencia Computacional.
- Polaridad Compuesta.
- Niveles Aceptables de Precisión.

### 4. Guardar el conjunto de datos limpio
Guardamos el conjunto de datos limpio en un nuevo archivo en formato CSV para facilitar el acceso y la reutilización.

In [31]:
# Los archivos se almacenan en local 
df_UserReviews.to_csv('user_reviews_cleaned.csv', index=False)